# UCN LD2 Calculations: Helical Groove as a Straight Tube - 2 Grooves

K. Augusto, J. Martin

June 9/20

Calculates the performance of a helical groove heat exchanger in the steady state operation of the LD2 thermosyphon.  Straight tube correlations are used for friction factors and Nusselt numbers.  The straight tube has a square profile and the appropriate hydraulic diameter formulae are used relative to the circular tube formulae.  Use of helical correlations is studied in another notebook. The dimensions of the groove were changed to have the same outlet temperature as one groove.

In [32]:
#!/usr/bin/python3

from math import *

from mpl_toolkits import mplot3d

from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from numpy import *

In [33]:
##############################################################
#General Constants Used
#############################################################
p_psi=20. # PSI, from CM.LD2.015.R2
p=p_psi*6894.76 # Pa
kt=0.104 # W/(m*K) from https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote641.pdf
T=Tin=23.4 # (K) inlet temp from CM.LD2.015.R2
Tw=20.7 # (K) temperature of cold wall 
mdot=0.004 # kg/s from CM.LD2.015.R2; based on the pressure head driven by (Delta)T=23.4-21.04=2.36K. If the outlet temperature doesn't reach 21.04 K (or less), the HEX doesn't work.
mu=3.5e-5 # Pa*s from https://nvlpubs.nist.gov/nistpubs/Legacy/TN/nbstechnicalnote641.pdf 
rho=163.0 # kg/m^3 from CRYOGENIC DATA BOOK by Dudley B. Chelton and Douglas B. Mann, pg 22
Cp=6565.4 # J/(kg*K) from CoolProp

##############################################################
#Geometry of HEX
#############################################################
L=10*0.0254 #m physical height of tube, from CM.LD2.015.R2
Ngrooves=2 # number of grooves, the optimal groove will be explored in another notebook 
D=4.76*0.0254 #m diameter of tube from CM.LD2.015.R2
wprime= 0.01 #m width of groove
uprime= 0.01 # m width between grooves
depth=0.01 # m depth of groove

The following diagram displays the unrolled helical HEX geometry that is used in the calculation.

<img src="helix-drawing-v2.png">

To find the pitch angle:

\begin{equation}
\alpha = \sin^{-1}\left(\frac{N(w'+u')}{\pi D}\right)
\end{equation}

where $w'$ is the width of the groove, $u'$ is the width of material between the grooves, $N$ is the number of grooves, and $D$ is the diameter of the Copper bar into which the helical grooves are cut.

In [34]:
sinalpha=(Ngrooves*(wprime + uprime))/(pi*D) #pitch angle
alpha=arcsin(sinalpha)
print('The pitch angle is %f radians' %alpha)

The pitch angle is 0.105506 radians


In [35]:
Lprime=L/sinalpha #m, length of wound groove
print('The length of the groove is %f m.' %Lprime)
appturns=Lprime/(pi*D) # note:  a small-angle approximation has been used here
print('Coiling around a Cu rod of diameter %f m would require approximately %f turns'%(D,appturns))
turns=L/(tan(alpha)*pi*D) # correct calculation with no small-angle approximation
print('Coiling around a Cu rod of diameter %f m would require %f turns'%(D,turns))

The length of the groove is 2.411928 m.
Coiling around a Cu rod of diameter 0.120904 m would require approximately 6.350000 turns
Coiling around a Cu rod of diameter 0.120904 m would require 6.314690 turns


The total flow area of all the helical channels combined is found from:

\begin{equation}
A = Nw'depth
\end{equation}

The total flow perimeter of all the helical channels combined is found from:

\begin{equation}
P = 2N(depth + w')
\end{equation}

In [36]:
ahelix=Ngrooves*wprime*depth # m^2, total flow area of all the grooves
print('The area of the helical fins is %f m^2.'%ahelix)
phelix=Ngrooves*(2*depth+2*wprime) # m, total flow perimeter of all the grooves 
print('The perimeter of the helical grooves is %f m.' %phelix)

The area of the helical fins is 0.000200 m^2.
The perimeter of the helical grooves is 0.080000 m.


The hydraluic diameter is found from Equation 6.11 from "Cryogenic Heat Transfer" by Randall F. Barron and Gregory F. Nellis (Barron):

\begin{equation}
D_h = \frac{4A}{P}
\end{equation}

The formula for $D_h$ is independent of the number of channels used, if $w'$ and $depth$ are kept constant.  The quantity $N$ cancels.

The Mass flux per unit area is found from:

\begin{equation}
G = \frac{\dot{m}}{A}
\end{equation}

The mass flux per unit area $G$ drops appropriately when the number of channels $N$ is increased, for fixed channel dimensions $w'$ and $depth$.

In [37]:
Dh=4*ahelix/phelix #m
print('Hydraulic diameter %f m'%Dh)
G=mdot/ahelix # (kg/(m^2*s)) mass flow rate per unit area
print('Mass flux per unit area is %f kg/(m^2*s)'%G)

Hydraulic diameter 0.010000 m
Mass flux per unit area is 20.000000 kg/(m^2*s)


The Reynolds Number is found from Equation 6.10 from Barron:

\begin{equation}
Re = \frac{D_hG}{\mu}
\end{equation}

In [38]:
Re=Dh*G/mu # dimensionless
print('The Reynolds number is %f'%Re)

The Reynolds number is 5714.285714


In order to calculate the heat transfer coefficient, the Nusselt number must be determined from correlations.  In the following calculation, we use the correlations found in Barron, and following the discussion of Example 6.1 in Chapter 6.

The Colburn factor is found from Equation 6.33 of Barron:

\begin{equation}
j_H = 0.023Re^{-1/5}B_1
\end{equation}

According to equation 6.34 of Barron, the factor $B_1$ is used for liquids only, as it is 1 for gases.  It relates to the viscosity ratio at the wall temperature relative the bulk liquid temperature, as explained in Equation 6.35 of Barron.  For the wall temperature a value of 20.7 K was used.  For the bulk liquid temperature a value was derived based on the inlet liquid temperature, which, as will be seen below was 23.4 K.  The viscosities were taken from "UCN LD2 Heat exchanger Design" by C. Marshall.  These were then used in the following formula which is Equation 6.35 of Barron.

\begin{equation}
B_1=1.174 \bigg( \frac{\mu_{ave}}{\mu_w} \bigg)^{0.14}
\end{equation}

As will be seen below, the viscosity ratio is very close to unity for the given parameters.  The main effect of including $B_1$ is then that the factor 1.174 is included, which inceases the Colburn J-factor $j_H$ by 14\%.


In [39]:
B1=1.174*((3.7e-5)/(3.68e-5))**(0.14) #viscosity taken from CM.LD2.015.R2
print('This is B1 %f.' %B1)
if Re < 3500 :
    print('It is laminar or in between')
elif Re > 3500 :
    jh=0.023*Re**(-0.2)*B1
    print('The Colburn factor for the turbulent flow is %f.' %jh)

This is B1 1.174891.
The Colburn factor for the turbulent flow is 0.004790.


The Prandlt Number is found from Equation 6.14 of Barron:

\begin{equation}
{\rm Pr} = \frac{\mu C_p}{k_t}
\end{equation}

For turbulent flow (${\rm Re}>3500$), the Nusselt Number is found from Equation 6.18 of Barron:

\begin{equation}
{\rm Nu} = j_H{\rm Re}{\rm Pr}^{1/3}
\end{equation}

In [40]:
Pr=(mu*Cp)/(kt) # dimentionless
print('The Prandtl Number is %f.'%Pr)
if Re < 3500 :
    Nu=4.8608
    print('Nu=4.8608 because the flow is laminar')
elif Re > 3500 :
    Nuturb=jh*Re*Pr**(1./3.)
    print('This is the turbulent Nusselt Number %f.' %Nuturb)

The Prandtl Number is 2.209510.
This is the turbulent Nusselt Number 35.650066.


The heat transfer coefficient is then found from Equation 6.17 from Barron:

\begin{equation}
h_c = \frac{{\rm Nu}k_t}{D_h}
\end{equation}

In [41]:
if Re < 3500 :
    hc=Nu*kt/Dh 
    print('The heat transfer coefficient for laminar flow is %f W/(m^2*K)'%hc)
elif Re > 3500 :
    hc=Nuturb*kt/Dh 
    print('The heat transfer coefficient for turbulent flow is %f W/(m^2*K)'%hc)

The heat transfer coefficient for turbulent flow is 370.760687 W/(m^2*K)


For turbulent flow, the friction factor is found Equation 6.32 from Barron:

\begin{equation}
f = 0.316Re^{(-1/4)}
\end{equation}

In [42]:
fRe=24.00*4
if Re < 2300 :
    f=fRe/Re
    #f=64/Re #assuming cicular tube
    print('The laminar friction factor is %f.' %f)
elif 3500 > Re > 2300 :
    f=1.2036*Re**(-0.416) #from vijayan
    print('The friction factor is in between laminar and turbulent')
elif Re > 3500 :
    f=0.316*Re**(-0.25)
    print('The turbulent friction factor is %f.' %f)

The turbulent friction factor is 0.036345.


The pressure drop is found from Equation 6.21 from Barron:

\begin{equation}
\Delta P = \frac{fL'G^2}{2D_h\rho}
\end{equation}

In [43]:
dp=(f*Lprime*G**2)/(Dh*2*rho) # (Pa) pressure drop

print('The pressure drop is %f Pa'%dp)

The pressure drop is 10.756061 Pa


The area of the wall transfering heat is found from:

\begin{equation}
A_w = NL'(w' + 2depth)
\end{equation}

In this calculation, possible fluid flow over the top of the $u'$ sections has been ignored.  Only heat transfer from within the ``U'' of the grooves has been included.

In [44]:
Aw=Ngrooves*(wprime+2*depth)*Lprime
print('Area of cold wall %f m^2'%Aw)

Area of cold wall 0.144716 m^2


The number of transfer units is found from Equation 6.41 of Barron:

\begin{equation}Ntu = \frac{A_w h_c}{\dot{m} C_p}
\end{equation}

In [45]:
Ntu=hc*Aw/(mdot*Cp)
print('The number of transfer units is %f'%Ntu)
print()

The number of transfer units is 2.043092



Given the inlet temperature $T_1=T_{\rm in}$ and the wall temperature $T_w$, the outlet temperature $T_2$ can be calculated from the number of transfer units $Ntu$ using Equation 6.42 of Barron as:
\begin{equation}
T_2=T_1-(T_1-T_w)(1-e^{-Ntu}).
\end{equation}


The total heat transfer rate may then be found using Equation 6.43 of Barron:
\begin{equation}
Q = \dot{m}C_p(T1-T2).
\end{equation}

In [46]:
T1=Tin
T2=T1-(T1-Tw)*(1-exp(-Ntu))
T2=Tw+(T1-Tw)*exp(-Ntu)

Qtotal=mdot*Cp*(T1-T2) # Eq. (6.43) of Barron

print('For inlet temperature %f K and wall temperature %f K'%(T1,Tw))
print('the outlet temperature is %f K'%T2)
print('and the total heat transfer rate is %f W'%Qtotal)
print()

For inlet temperature 23.400000 K and wall temperature 20.700000 K
the outlet temperature is 21.049994 K
and the total heat transfer rate is 61.714930 W

